## Autoencoder transformation (encode-decode)

Considering a dataset with $p$ numerical attributes. 

The goal of the autoencoder is to reduce the dimension of $p$ to $k$, such that these $k$ attributes are enough to recompose the original $p$ attributes. However from the $k$ dimensionals the data is returned back to $p$ dimensions. The higher the quality of autoencoder the similiar is the output from the input. 

In [1]:
# DAL ToolBox
# version 1.01.727

source("https://raw.githubusercontent.com/cefet-rj-dal/daltoolbox/main/jupyter.R")

#loading DAL
load_library("daltoolbox")

Loading required package: daltoolbox

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘daltoolbox’


The following object is masked from ‘package:base’:

    transform




### dataset for example 

In [2]:
data(sin_data)

sw_size <- 5
ts <- ts_data(sin_data$y, sw_size)

ts_head(ts)


t4,t3,t2,t1,t0
0.0000000,0.2474040,0.4794255,0.6816388,0.8414710
0.2474040,0.4794255,0.6816388,0.8414710,0.9489846
0.4794255,0.6816388,0.8414710,0.9489846,0.9974950
0.6816388,0.8414710,0.9489846,0.9974950,0.9839859
0.8414710,0.9489846,0.9974950,0.9839859,0.9092974
0.9489846,0.9974950,0.9839859,0.9092974,0.7780732


### applying data normalization

In [3]:
preproc <- ts_norm_gminmax()
preproc <- fit(preproc, ts)
ts <- transform(preproc, ts)

ts_head(ts)

t4,t3,t2,t1,t0
0.5004502,0.6243512,0.7405486,0.8418178,0.9218625
0.6243512,0.7405486,0.8418178,0.9218625,0.9757058
0.7405486,0.8418178,0.9218625,0.9757058,1.0000000
0.8418178,0.9218625,0.9757058,1.0000000,0.9932346
0.9218625,0.9757058,1.0000000,0.9932346,0.9558303
0.9757058,1.0000000,0.9932346,0.9558303,0.8901126


### spliting into training and test

In [4]:
samp <- ts_sample(ts, test_size = 10)
train <- as.data.frame(samp$train)
test <- as.data.frame(samp$test)

### creating autoencoder
Reduce from 5 to 3 dimensions

In [5]:
auto <- autoenc_encode_decode(5, 3)

auto <- fit(auto, train)

### testing autoencoder
presenting the original test set and display encoding

In [6]:
print(head(test))
result <- transform(auto, test)
print(head(result))

         t4        t3        t2        t1        t0
1 0.7258342 0.8294719 0.9126527 0.9702046 0.9985496
2 0.8294719 0.9126527 0.9702046 0.9985496 0.9959251
3 0.9126527 0.9702046 0.9985496 0.9959251 0.9624944
4 0.9702046 0.9985496 0.9959251 0.9624944 0.9003360
5 0.9985496 0.9959251 0.9624944 0.9003360 0.8133146
6 0.9959251 0.9624944 0.9003360 0.8133146 0.7068409
          [,1]      [,2]      [,3]      [,4]      [,5]
[1,] 0.7256835 0.8295398 0.9124184 0.9693199 0.9961401
[2,] 0.8272868 0.9130398 0.9703655 0.9993480 0.9953367
[3,] 0.9097142 0.9712421 0.9992328 0.9982169 0.9634869
[4,] 0.9693188 1.0005627 0.9958416 0.9633672 0.8987426
[5,] 0.9977111 0.9974224 0.9616431 0.9008309 0.8118427
[6,] 0.9965426 0.9642960 0.8998429 0.8147668 0.7067665
